In [14]:
import tkinter as tk
from tkinter import messagebox
from tkinter import ttk
import pandas as pd
import zipfile
import os

# Path to your zip file
zip_file_path = 'archive.zip'
extract_dir = 'extracted_files/'

# Extract the CSV files from the zip archive
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# Step 1: Load all CSV files into a single DataFrame
csv_files = [f for f in os.listdir(extract_dir) if f.endswith('.csv')]
data_frames = []
for file in csv_files:
    file_path = os.path.join(extract_dir, file)
    df = pd.read_csv(file_path)
    data_frames.append(df)

# Combine all data into a single DataFrame
combined_df = pd.concat(data_frames, ignore_index=True)

# Step 2: Filter relevant columns
relevant_columns = ['movie_id', 'movie_name', 'genre', 'rating', 'director', 'runtime', 'description']
movies_df = combined_df[relevant_columns]

# Function to apply themed styles
def apply_styles():
    style = ttk.Style()
    style.theme_use("clam")  # Modern theme
    style.configure("TButton", font=("Arial", 12), padding=6, background="#e0e0e0", foreground="#000", relief="flat")
    style.configure("TLabel", font=("Arial", 12), background="#f5f5f5", foreground="#333")
    style.configure("Treeview", font=("Arial", 12), background="#ffffff", foreground="#333", fieldbackground="#ffffff")
    style.configure("Treeview.Heading", font=("Arial", 14, "bold"), background="#424242", foreground="white")
    style.map("TButton", background=[("active", "#d6d6d6")], foreground=[("active", "#000")])

# Functions for login, logout, and genre-based filtering
def login():
    username = username_entry.get()
    password = password_entry.get()
    if password == "root":
        login_window.destroy()  # Close the login window
        create_movie_recommendation_page(username)  # Open the main movie recommendation page
    else:
        messagebox.showerror("Login Failed", "Invalid Password!")

def logout():
    if messagebox.askyesno("Confirm Logout", "Are you sure you want to log out?"):
        window.destroy()
        relaunch_login()

def show_movies_by_genre(genre):
    filtered_movies = movies_df[movies_df['genre'].str.contains(genre, case=False, na=False)]
    sorted_movies = filtered_movies.nlargest(20, 'rating')

    if sorted_movies.empty:
        messagebox.showinfo("No Movies Found", f"No movies found for the genre: {genre}")
        return

    movie_treeview.delete(*movie_treeview.get_children())
    for _, row in sorted_movies.iterrows():
        movie_treeview.insert("", "end", values=(row['movie_name'], row['rating']))

def show_movie_details(movie_name):
    movie_details = movies_df[movies_df['movie_name'] == movie_name].iloc[0]
    details = f"Movie Name: {movie_details['movie_name']}\n"
    details += f"Director: {movie_details['director']}\n"
    details += f"Rating: {movie_details['rating']}\n"
    details += f"Runtime: {movie_details['runtime']} mins\n"
    details += f"Description: {movie_details['description']}\n"

    details_text.config(state=tk.NORMAL)
    details_text.delete(1.0, tk.END)
    details_text.insert(tk.END, details)
    details_text.config(state=tk.DISABLED)

def clear_movie_list_and_details():
    movie_treeview.delete(*movie_treeview.get_children())
    details_text.config(state=tk.NORMAL)
    details_text.delete(1.0, tk.END)
    details_text.insert(tk.END, "Select a genre to view movies.")
    details_text.config(state=tk.DISABLED)

def create_movie_recommendation_page(username):
    global window, movie_treeview, details_text

    # Main window
    window = tk.Tk()
    window.title("Movie Recommendation System")
    window.geometry("1000x700")
    window.config(bg="#f5f5f5")
    apply_styles()

    # Header
    header_frame = tk.Frame(window, bg="#424242")
    header_frame.pack(fill=tk.X)

    heading_label = tk.Label(header_frame, text="Movie Recommendation System", font=("Arial", 24, "bold"), bg="#424242", fg="white")
    heading_label.pack(pady=10)

    login_user_label = tk.Label(header_frame, text=f"Welcome, {username}!", font=("Arial", 16), bg="#424242", fg="white")
    login_user_label.pack(side=tk.LEFT, padx=10)

    logout_button = ttk.Button(header_frame, text="Logout", command=logout)
    logout_button.pack(side=tk.RIGHT, padx=10)

    # Genre Selection
    genres = ['Action', 'Adventure', 'Animation', 'Biography', 'Crime', 'Family', 'Fantasy', 'Film-Noir',
              'History', 'Horror', 'Mystery', 'Romance', 'Sci-Fi', 'Sports', 'Thriller', 'War']

    genre_frame = tk.Frame(window, bg="#f5f5f5")
    genre_frame.pack(pady=20)

    tk.Label(genre_frame, text="Select a Genre", font=("Arial", 16), bg="#f5f5f5", fg="#333").pack()

    for genre in genres:
        ttk.Button(genre_frame, text=genre, command=lambda g=genre: show_movies_by_genre(g)).pack(side=tk.LEFT, padx=5)

    # Movie List
    tree_frame = tk.Frame(window)
    tree_frame.pack(pady=20)

    movie_treeview = ttk.Treeview(tree_frame, columns=("Movie Name", "Rating"), show="headings", height=10)
    movie_treeview.heading("Movie Name", text="Movie Name")
    movie_treeview.heading("Rating", text="Rating")
    movie_treeview.pack(fill=tk.BOTH, expand=True)

    # Movie Details
    details_frame = tk.Frame(window, bg="#f5f5f5")
    details_frame.pack(pady=20)

    details_text = tk.Text(details_frame, height=10, width=80, wrap=tk.WORD, font=("Arial", 12), padx=10, pady=5)
    details_text.pack()
    details_text.insert(tk.END, "Select a movie to see details.")
    details_text.config(state=tk.DISABLED)

    # Clear Button
    ttk.Button(window, text="Clear", command=clear_movie_list_and_details).pack(pady=10)

    # Event Binding
    movie_treeview.bind("<Double-1>", lambda e: show_movie_details(movie_treeview.item(movie_treeview.selection()[0], "values")[0]))

    window.mainloop()

# Login Page
def relaunch_login():
    global login_window, username_entry, password_entry

    login_window = tk.Tk()
    login_window.title("Login")
    login_window.geometry("400x300")
    login_window.config(bg="#eeeeee")
    apply_styles()

    tk.Label(login_window, text="Login to Movie Recommendation System", font=("Arial", 16, "bold"), bg="#eeeeee", fg="#333").pack(pady=20)

    tk.Label(login_window, text="Username:", font=("Arial", 12), bg="#eeeeee", fg="#333").pack(pady=10)
    username_entry = ttk.Entry(login_window, font=("Arial", 12))
    username_entry.pack(pady=10)

    tk.Label(login_window, text="Password:", font=("Arial", 12), bg="#eeeeee", fg="#333").pack(pady=10)
    password_entry = ttk.Entry(login_window, font=("Arial", 12), show="*")
    password_entry.pack(pady=10)

    ttk.Button(login_window, text="Login", command=login).pack(pady=20)

    login_window.mainloop()

# Start Application
relaunch_login()
